# Set-up notebook environment
## NOTE: Use a QIIME2 kernel

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy
from scipy import stats
import matplotlib.pyplot as plt
import re
from pandas import *
import matplotlib.pyplot as plt
%matplotlib inline
from qiime2.plugins import feature_table
from qiime2 import Artifact
from qiime2 import Metadata
import biom
from biom.table import Table
from qiime2.plugins import diversity
from scipy.stats import ttest_ind
from scipy.stats.stats import pearsonr
%config InlineBackend.figure_formats = ['svg']
from qiime2.plugins.feature_table.methods import relative_frequency
import biom
import qiime2 as q2
import os
import math


# Import sample metadata

In [2]:
meta = q2.Metadata.load('/Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/sample_metadata/12201_metadata.txt').to_dataframe()


Separate round 1 and round 2 and exclude round 1 Zymo, Homebrew, and MagMAX Beta

In [4]:
meta_r1 = meta[meta['round'] == 1]
meta_clean_r1_1 = meta_r1[meta_r1['extraction_kit'] != 'Zymo MagBead']
meta_clean_r1_2 = meta_clean_r1_1[meta_clean_r1_1['extraction_kit'] != 'Homebrew']
meta_clean_r1 = meta_clean_r1_2[meta_clean_r1_2['extraction_kit'] != 'MagMax Beta']
meta_clean_r2 = meta[meta['round'] == 2]


Remove PowerSoil samples from each round - these samples will be used as the baseline 

In [5]:
meta_clean_r1_noPS = meta_clean_r1[meta_clean_r1['extraction_kit'] != 'PowerSoil']
meta_clean_r2_noPS = meta_clean_r2[meta_clean_r2['extraction_kit'] != 'PowerSoil']


Create tables including only round 1 or round 2 PowerSoil samples

In [6]:
meta_clean_r1_onlyPS = meta_clean_r1[meta_clean_r1['extraction_kit'] == 'PowerSoil']
meta_clean_r2_onlyPS = meta_clean_r2[meta_clean_r2['extraction_kit'] == 'PowerSoil']


Merge PowerSoil samples from round 2 with other samples from round 1, and vice versa - this will allow us to get the correlations between the two rounds of PowerSoil

In [7]:
meta_clean_r1_with_r2_PS = pd.concat([meta_clean_r1_noPS, meta_clean_r2_onlyPS])
meta_clean_r2_with_r1_PS = pd.concat([meta_clean_r2_noPS, meta_clean_r1_onlyPS])


## Collapse feature-table to the desired level (e.g., genus)

16S

In [ ]:
qiime taxa collapse \
  --i-table /Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/data/16S/10_filtered_data/dna_bothPS_16S_deblur_biom_lod_noChl_noMit_sepp_gg_noNTCs_noMock.qza \
  --i-taxonomy /Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/data/16S/06_taxonomy/dna_all_16S_deblur_seqs_taxonomy_silva138.qza \
  --p-level 6 \
  --o-collapsed-table /Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/data/16S/10_filtered_data/dna_bothPS_16S_deblur_biom_lod_noChl_noMit_sepp_gg_noNTCs_noMock_taxa_collapse_genus.qza

qiime feature-table summarize \
  --i-table /Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/data/16S/10_filtered_data/dna_bothPS_16S_deblur_biom_lod_noChl_noMit_sepp_gg_noNTCs_noMock_taxa_collapse_genus.qza \
  --o-visualization /Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/data/16S/10_filtered_data/dna_bothPS_16S_deblur_biom_lod_noChl_noMit_sepp_gg_noNTCs_noMock_taxa_collapse_genus.qzv

# There are 846 samples and 1660 features


ITS

In [ ]:
qiime taxa collapse \
  --i-table /Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/data/ITS/08_filtered_data/dna_bothPS_ITS_deblur_biom_lod_noNTCs_noMock.qza \
  --i-taxonomy /Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/data/ITS/06_taxonomy/dna_all_ITS_deblur_seqs_taxonomy_unite8.qza \
  --p-level 6 \
  --o-collapsed-table /Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/data/ITS/08_filtered_data/dna_bothPS_ITS_deblur_biom_lod_noNTCs_noMock_taxa_collapse_genus.qza

qiime feature-table summarize \
  --i-table /Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/data/ITS/08_filtered_data/dna_bothPS_ITS_deblur_biom_lod_noNTCs_noMock_taxa_collapse_genus.qza \
  --o-visualization /Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/data/ITS/08_filtered_data/dna_bothPS_ITS_deblur_biom_lod_noNTCs_noMock_taxa_collapse_genus.qzv

# There are 978 samples and 791 features


Shotgun

In [ ]:
qiime taxa collapse \
  --i-table /Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/data/shotgun/03_filtered_data/dna_bothPS_shotgun_woltka_wol_biom_noNTCs_noMock.qza \
  --i-taxonomy /Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/data/shotgun/wol_taxonomy.qza \
  --p-level 6 \
  --o-collapsed-table /Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/data/shotgun/03_filtered_data/dna_bothPS_shotgun_woltka_wol_biom_noNTCs_noMock_taxa_collapse_genus.qza

qiime feature-table summarize \
  --i-table /Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/data/shotgun/03_filtered_data/dna_bothPS_shotgun_woltka_wol_biom_noNTCs_noMock_taxa_collapse_genus.qza \
  --o-visualization /Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/data/shotgun/03_filtered_data/dna_bothPS_shotgun_woltka_wol_biom_noNTCs_noMock_taxa_collapse_genus.qzv

# There are 1044 samples and 2060 features


# Import feature-tables

In [8]:
dna_bothPS_16S_genus_qza = q2.Artifact.load('/Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/data/16S/10_filtered_data/dna_bothPS_16S_deblur_biom_lod_noChl_noMit_sepp_gg_noNTCs_noMock_taxa_collapse_genus.qza')
dna_bothPS_ITS_genus_qza = q2.Artifact.load('/Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/data/ITS/08_filtered_data/dna_bothPS_ITS_deblur_biom_lod_noNTCs_noMock_taxa_collapse_genus.qza')
dna_bothPS_shotgun_genus_qza = q2.Artifact.load('/Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/data/shotgun/03_filtered_data/dna_bothPS_shotgun_woltka_wol_biom_noNTCs_noMock_taxa_collapse_genus.qza')


# Convert QZA to a Pandas DataFrame

In [9]:
dna_bothPS_16S_genus_df = dna_bothPS_16S_genus_qza.view(pd.DataFrame)
dna_bothPS_ITS_genus_df = dna_bothPS_ITS_genus_qza.view(pd.DataFrame)
dna_bothPS_shotgun_genus_df = dna_bothPS_shotgun_genus_qza.view(pd.DataFrame)


# Melt dataframes

In [10]:
dna_bothPS_16S_genus_df_melt = dna_bothPS_16S_genus_df.unstack()
dna_bothPS_ITS_genus_df_melt = dna_bothPS_ITS_genus_df.unstack()
dna_bothPS_shotgun_genus_df_melt = dna_bothPS_shotgun_genus_df.unstack()

dna_bothPS_16S_genus = pd.DataFrame(dna_bothPS_16S_genus_df_melt)
dna_bothPS_ITS_genus = pd.DataFrame(dna_bothPS_ITS_genus_df_melt)
dna_bothPS_shotgun_genus = pd.DataFrame(dna_bothPS_shotgun_genus_df_melt)


In [11]:
dna_bothPS_16S_genus.reset_index(inplace=True)
dna_bothPS_16S_genus.rename(columns={'level_0':'taxa','level_1':'sample',0:'counts'}, inplace=True)

dna_bothPS_ITS_genus.reset_index(inplace=True)
dna_bothPS_ITS_genus.rename(columns={'level_0':'taxa','level_1':'sample',0:'counts'}, inplace=True)

dna_bothPS_shotgun_genus.reset_index(inplace=True)
dna_bothPS_shotgun_genus.rename(columns={'level_0':'taxa','level_1':'sample',0:'counts'}, inplace=True)


# Wrangle data into long form for each kit

Wrangle metadata

In [12]:
# Create empty list of extraction kit IDs
ext_kit_levels = [] 


# Create empty list of metadata subsets based on levels of variable of interest
ext_kit = [] 


# Create empty list of baseline samples for each subset
bl = []


# Populate lists with round 1 data
for ext_kit_level, ext_kit_level_df in meta_clean_r1_with_r2_PS.groupby('extraction_kit_round'):
    ext_kit.append(ext_kit_level_df)
    
    powersoil_r1_bl = meta_clean_r1_onlyPS[meta_clean_r1_onlyPS.extraction_kit_round == 'PowerSoil r1']
    bl.append(powersoil_r1_bl)
    
    ext_kit_levels.append(ext_kit_level)
    
    print('Gathered data for',ext_kit_level)


# Populate lists with round 2 data
for ext_kit_level, ext_kit_level_df in meta_clean_r2_with_r1_PS.groupby('extraction_kit_round'):
    ext_kit.append(ext_kit_level_df)

    powersoil_r2_bl = meta_clean_r2_onlyPS[meta_clean_r2_onlyPS['extraction_kit_round'] == 'PowerSoil r2']
    bl.append(powersoil_r2_bl)

    ext_kit_levels.append(ext_kit_level)

    print('Gathered data for',ext_kit_level)


# Create empty list for concatenated subset-baseline datasets
subsets_w_bl = {}


# Populate list with subset-baseline data
for ext_kit_level, ext_kit_df, ext_kit_bl in zip(ext_kit_levels, ext_kit, bl):    

        new_df = pd.concat([ext_kit_bl,ext_kit_df]) 
        subsets_w_bl[ext_kit_level] = new_df
        
        print('Merged data for',ext_kit_level)
   

Gathered data for Norgen
Gathered data for PowerSoil Pro
Gathered data for PowerSoil r2
Gathered data for MagMAX Microbiome
Gathered data for NucleoMag Food
Gathered data for PowerSoil r1
Gathered data for Zymo MagBead
Merged data for Norgen
Merged data for PowerSoil Pro
Merged data for PowerSoil r2
Merged data for MagMAX Microbiome
Merged data for NucleoMag Food
Merged data for PowerSoil r1
Merged data for Zymo MagBead


16S

In [200]:
list_of_lists = []

for key, value in subsets_w_bl.items():
    
    string =  ''.join(key)
    
    #merge metadata subsets with baseline with taxonomy
    meta_16S_genera = pd.merge(value, dna_bothPS_16S_genus, left_index=True, right_on='sample')

    #create new column 
    meta_16S_genera['taxa_subject'] = meta_16S_genera['taxa'] + meta_16S_genera['host_subject_id']

    #subtract out duplicates and pivot
    meta_16S_genera_clean = meta_16S_genera.drop_duplicates(subset = ['taxa_subject', 'extraction_kit_round'], keep = 'first')
    meta_16S_genera_pivot = meta_16S_genera_clean.pivot(index='taxa_subject', columns='extraction_kit_round', values='counts')
    meta_16S_genera_pivot_clean = meta_16S_genera_pivot.dropna()
    
    # Export dataframe to file
    meta_16S_genera_pivot_clean.to_csv('/Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/results/feature_abundance_correlation_images/table_correlation_16S_genera_%s.txt'%string,
         sep = '\t',
         index = False)
   

ITS

In [13]:
list_of_lists = []

for key, value in subsets_w_bl.items():
    
    string =  ''.join(key)
    
    #merge metadata subsets with baseline with taxonomy
    meta_ITS_genera = pd.merge(value, dna_bothPS_ITS_genus, left_index=True, right_on='sample')

    #create new column 
    meta_ITS_genera['taxa_subject'] = meta_ITS_genera['taxa'] + meta_ITS_genera['host_subject_id']

    #subtract out duplicates and pivot
    meta_ITS_genera_clean = meta_ITS_genera.drop_duplicates(subset = ['taxa_subject', 'extraction_kit_round'], keep = 'first')
    meta_ITS_genera_pivot = meta_ITS_genera_clean.pivot(index='taxa_subject', columns='extraction_kit_round', values='counts')
    meta_ITS_genera_pivot_clean = meta_ITS_genera_pivot.dropna()
    
    # Export dataframe to file
    meta_ITS_genera_pivot_clean.to_csv('/Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/results/feature_abundance_correlation_images/table_correlation_ITS_genera_%s.txt'%string,
         sep = '\t',
         index = False)
   

Shotgun

In [14]:
list_of_lists = []

for key, value in subsets_w_bl.items():
    
    string =  ''.join(key)
    
    #merge metadata subsets with baseline with taxonomy
    meta_shotgun_genera = pd.merge(value, dna_bothPS_shotgun_genus, left_index=True, right_on='sample')

    #create new column 
    meta_shotgun_genera['taxa_subject'] = meta_shotgun_genera['taxa'] + meta_shotgun_genera['host_subject_id']

    #subtract out duplicates and pivot
    meta_shotgun_genera_clean = meta_shotgun_genera.drop_duplicates(subset = ['taxa_subject', 'extraction_kit_round'], keep = 'first')
    meta_shotgun_genera_pivot = meta_shotgun_genera_clean.pivot(index='taxa_subject', columns='extraction_kit_round', values='counts')
    meta_shotgun_genera_pivot_clean = meta_shotgun_genera_pivot.dropna()
    
    # Export dataframe to file
    meta_shotgun_genera_pivot_clean.to_csv('/Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/results/feature_abundance_correlation_images/table_correlation_shotgun_genera_%s.txt'%string,
         sep = '\t',
         index = False)
   

# Code below is not used
## NOTE: The first cell was originally appended to the cell above

In [ ]:
 # check pearson correlation
    x = meta_16S_genera_pivot_clean.iloc[:,1]
    y = meta_16S_genera_pivot_clean[key]
    corr = stats.pearsonr(x, y)
    int1, int2 = corr
    corr_rounded = round(int1, 2)
    corr_str = str(corr_rounded)
    x_key = key[0]
    y_key = key[1]
    
    list1 = []
    
    list1.append(corr_rounded)
    list1.append(key)
    list_of_lists.append(list1)
        

In [ ]:
list_of_lists

In [154]:
df = pd.DataFrame(list_of_lists, columns = ['Correlation', 'Extraction kit']) 


In [198]:
df.to_csv('/Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/results/feature_abundance_correlation_images/table_correlations_16S_genera.txt',
         sep = '\t',
         index = False)


In [ ]:
splot = sns.catplot(y="Correlation", 
                    x="Extraction kit", 
                    hue= "Extraction kit", 
                    kind='bar',
                    data=df,
                   dodge = False)
splot.set(ylim=(0, 1))
plt.xticks(rotation=45,
          horizontalalignment='right')

#new_labels = ['−20C','−20C after 1 week', '4C','Ambient','Freeze-thaw','Heat']
#for t, l in zip(splot._legend.texts, new_labels):
#    t.set_text(l)
        
splot.savefig('correlation_16S_genera.png')
splot.savefig('correlation_16S_genera.svg', format='svg', dpi=1200)


### Individual correlation plots 

In [ ]:
for key, value in subsets_w_bl.items():
    
    string =  ''.join(key)
    
    #merge metadata subsets with baseline with taxonomy
    meta_16S_genera = pd.merge(value, dna_bothPS_16S_genus, left_index=True, right_on='sample')

    #create new column 
    meta_16S_genera['taxa_subject'] = meta_16S_genera['taxa'] + meta_16S_genera['host_subject_id']

    #subtract out duplicates and pivot
    meta_16S_genera_clean = meta_16S_genera.drop_duplicates(subset = ['taxa_subject', 'extraction_kit_round'], keep = 'first')
    meta_16S_genera_pivot = meta_16S_genera_clean.pivot(index='taxa_subject', columns='extraction_kit_round', values='counts')
    meta_16S_genera_pivot_clean = meta_16S_genera_pivot.dropna()

    # check pearson correlation
    x = meta_16S_genera_pivot_clean.iloc[:,1]
    y = meta_16S_genera_pivot_clean[key]
    corr = stats.pearsonr(x, y)
    int1, int2 = corr
    corr_rounded = round(int1, 2)
    corr_str = str(corr_rounded)
    
    #make correlation plots
    meta_16S_genera_pivot_clean['x1'] = meta_16S_genera_pivot_clean.iloc[:,1]
    meta_16S_genera_pivot_clean['y1'] = meta_16S_genera_pivot_clean.iloc[:,0]
    ax=sns.lmplot(x='x1',
                  y='y1',
                  data=meta_16S_genera_pivot_clean, 
                  height=3.8)
    ax.set(yscale='log')
    ax.set(xscale='log')
    ax.set(xlabel='PowerSoil', ylabel=key)
    #plt.xlim(0.00001, 10000000)
    #plt.ylim(0.00001, 10000000)
    plt.title(string + ' (%s)' %corr_str)

    ax.savefig('/Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/results/feature_abundance_correlation_images/figure_scatter_correlation_16S_genera_%s.png'%string)
    ax.savefig('/Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/results/feature_abundance_correlation_images/figure_scatter_correlation_16S_genera_%s.svg'%string, format='svg',dpi=1200)
    